<a href="https://colab.research.google.com/github/henricobela/henricobela.github.io/blob/main/Checkpoint_Cyber_v6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install -q streamlit

In [10]:
!pip install -q streamlit-ace

In [11]:
!pip install -q fuzzywuzzy[speedup]

In [12]:
!pip install -q pyngrok

In [13]:
!streamlit run app.py &>/dev/null&

In [14]:
from pyngrok import ngrok

In [60]:
%%writefile app.py  
import streamlit as st            #importando streamlit e setando como st
import re                         #importando o regex
from fuzzywuzzy import process    #importando process do fuzzy
import json                       #importando json
import requests                   #importando requests


def main():                                                                       #função que cria webpage
  nameDataJson = requests.get("https://cspinheiro.github.io/interpol.json")       #consome json dos nomes da interpol e guarda na variavel nameDataJson
  fraDataJson = requests.get("https://henricobela.github.io/fra.json")            #consome json da França com numeros de passaportes e salva na variavel fraDataJson 

  def verifyNamesInterpol(names):
    invalidNames = nameDataJson.json()["interpol"]
    invalid = []

    extraction = process.extract(names, invalidNames)

    for x in extraction:
      if x[1] >= 80:
        invalid.append(x)
      if len(invalid) == 0:      
        return "Allow"
      return "Deny"

  def verifyPassportNmbr(passportNmbr):
    match = re.search("(?i)(fra)(\d+)", passportNmbr.lower())

    if match:
      dict_fra = fraDataJson.json()["fra"]
      for item in dict_fra:
        for value in item.values():
          if str(passportNmbr) == "fra"+str(value):
            return "Deny"
      return "Allow"
 
  def verifyPerson(name, passportNmbr):
    person = verifyNamesInterpol(name)
    passport = verifyPassportNmbr(passportNmbr)

    if passport == "Deny" or person == "Deny":
      st.error("Você não pode entrar no país")
      return
    st.success("Voce pode entrar no país")
    return

  st.set_page_config(page_title="France Airport", layout="centered")
  st.title("Imigrate Verification")
  form = st.form(key="form")

  with form:
    ownerName = st.text_input('Nome do Indíviduo:')
    passportNumber = st.text_input('Número do Passaporte:')
    submit = st.form_submit_button(label="SEND")

  if submit:
    if not ownerName or not passportNumber:
      st.warning("Digite todos os campos por favor...")
    else:
      verifyPerson(ownerName, passportNumber)

if __name__=='__main__': 
  main()

Overwriting app.py


In [61]:
!streamlit run app.py & npx localtunnel --port 8501

2022-04-28 18:36:00.501 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.125.241.38:8501

npx: installed 22 in 3.088s
your url is: https://modern-octopus-31.loca.lt
  Stopping...
^C
